# Setup

In [ ]:
# %pip install plotly
# %pip install protobuf==3.20.0
# %pip install circuitsvis

from transformer_lens.cautils.notebook import *

from transformer_lens.rs.callum2.ioi_and_bos.ioi_functions import (
    attn_scores_as_linear_func_of_keys,
    attn_scores_as_linear_func_of_queries,
    get_attn_scores_as_linear_func_of_queries_for_histogram,
    get_attn_scores_as_linear_func_of_keys_for_histogram,
    decompose_attn_scores,
    plot_contribution_to_attn_scores,
    project,
    decompose_attn_scores_full,
    create_fucking_massive_plot_1,
    create_fucking_massive_plot_2,
    get_nonspace_name_tokenIDs,
    get_nonspace_name_tokenIDs,
    get_lowercase_name_tokenIDs,
)
from transformer_lens.rs.callum2.utils import (
    get_effective_embedding,
)

clear_output()

In [ ]:
model = HookedTransformer.from_pretrained(
    "gpt2-small",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    # refactor_factored_attn_matrices=True,
)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)

clear_output()

In [ ]:
effective_embeddings = get_effective_embedding(model) 

W_U = model.W_U
W_EE = effective_embeddings["W_E (including MLPs)"]
W_EE_subE = effective_embeddings["W_E (only MLPs)"]

# Patching positions

Below, I'm going to perform a patching experiment where the residual stream values for `key_input` before heads in layer 10 are replaced with their values on the flipped dataset: where the token identities are swapped around. I expect this not to completely reverse the effect, which would be a sign that there's positional shit going on that we just failed to account for (maybe because it was mediated via the MLPs).